In [1]:
from flask import Flask, render_template, url_for
import flask_sqlalchemy
import json

app = Flask(__name__)
@app.route('/')
def home_page():
    with open('posts.json', 'r') as file:
        posts = json.load(file)["posts"]
        
    
    return render_template('index.html',posts_list = posts)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Mar/2023 23:35:41] "GET / HTTP/1.1" 200 -
